In [0]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType
from pyspark.sql import Row

In [0]:
schema=StructType([\
    StructField('id',IntegerType()),\
    StructField('name',StringType()),\
    StructField('company',StringType()),\
    StructField('domain',StringType())])
row1=Row(1,'lokesh','HCL','RPA')
row2=Row(2,'GOPAL','TCS','MEDICAL')
row3=Row(3,'SUHAIL','HCL','RPA')
row4=Row(4,'ESWAR','INFOSYS','APPS')
row5=Row(5,'RAMU','HCL','RPA')
row6=Row(6,'MANOJ','TCS','APPS')
data=[row1,row2,row3,row4,row5,row6]
df=spark.createDataFrame(data,schema)
display(df)

id,name,company,domain
1,lokesh,HCL,RPA
2,GOPAL,TCS,MEDICAL
3,SUHAIL,HCL,RPA
4,ESWAR,INFOSYS,APPS
5,RAMU,HCL,RPA
6,MANOJ,TCS,APPS


In [0]:
df.write.parquet('/FileStore/json/employees',mode='overwrite',partitionBy=['company','domain'])

In [0]:
df1=spark.read.parquet('/FileStore/json/employees').show()

+---+------+-------+-------+
| id|  name|company| domain|
+---+------+-------+-------+
|  1|lokesh|    HCL|    RPA|
|  3|SUHAIL|    HCL|    RPA|
|  4| ESWAR|INFOSYS|   APPS|
|  2| GOPAL|    TCS|MEDICAL|
|  6| MANOJ|    TCS|   APPS|
|  5|  RAMU|    HCL|    RPA|
+---+------+-------+-------+



In [0]:
spark.read.parquet('/FileStore/json/employees/company=HCL').show()

+---+------+------+
| id|  name|domain|
+---+------+------+
|  1|lokesh|   RPA|
|  3|SUHAIL|   RPA|
|  5|  RAMU|   RPA|
+---+------+------+



In [0]:
spark.read.parquet('/FileStore/json/employees/company=TCS/domain=MEDICAL').show()

+---+-----+
| id| name|
+---+-----+
|  2|GOPAL|
+---+-----+



In [0]:
# Convert json string to MapType or StructType
#   supose we have string in json format , then we cannot access keys or values from it because it is in string format so that we should convert json string to MapType pr StructType for that we use from_json

In [0]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import StringType,MapType

In [0]:
data=[(1,'lokesh','{"telugu":"25","english":"24"}')]
schema=['id','name','marks']
dfr=spark.createDataFrame(data,schema)
dfr.printSchema()
display(dfr)

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- marks: string (nullable = true)



id,name,marks
1,lokesh,"{""telugu"":""25"",""english"":""24""}"


In [0]:
Mapschema=MapType(keyType=StringType(),valueType=StringType())
dfr1=dfr.withColumn('Marks',from_json(dfr.marks,Mapschema))
dfr1.printSchema()
dfr1.show(truncate=False)


root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- Marks: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+---+------+-----------------------------+
|id |name  |Marks                        |
+---+------+-----------------------------+
|1  |lokesh|{telugu -> 25, english -> 24}|
+---+------+-----------------------------+



In [0]:
dfr1.withColumn('Telugu',dfr1.Marks.telugu).show()

+---+------+--------------------+------+
| id|  name|               Marks|Telugu|
+---+------+--------------------+------+
|  1|lokesh|{telugu -> 25, en...|    25|
+---+------+--------------------+------+



In [0]:
# convert into struct type
structschema=StructType([\
    StructField('telugu',StringType()),\
    StructField('english',StringType())])
dfr2=dfr.withColumn('structschema(marks)',from_json(dfr.marks,structschema))
dfr2.printSchema()
dfr2.show()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- marks: string (nullable = true)
 |-- structschema(marks): struct (nullable = true)
 |    |-- telugu: string (nullable = true)
 |    |-- english: string (nullable = true)

+---+------+--------------------+-------------------+
| id|  name|               marks|structschema(marks)|
+---+------+--------------------+-------------------+
|  1|lokesh|{"telugu":"25","e...|           {25, 24}|
+---+------+--------------------+-------------------+



In [0]:
from pyspark.sql.functions import col

In [0]:
dfr2.withColumn('telugu',col('structschema(marks)').telugu).show()

+---+------+--------------------+-------------------+------+
| id|  name|               marks|structschema(marks)|telugu|
+---+------+--------------------+-------------------+------+
|  1|lokesh|{"telugu":"25","e...|           {25, 24}|    25|
+---+------+--------------------+-------------------+------+



In [0]:
dfr1.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- Marks: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [0]:
from pyspark.sql.functions import to_json

In [0]:
# we try to convert this into json string bu using to_json
dfr3=dfr1.withColumn('jsonmarks',to_json(dfr1.Marks))
dfr3.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- Marks: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- jsonmarks: string (nullable = true)



In [0]:
# json_tuple used to query or extract fields or properties from json string
from pyspark.sql.functions import json_tuple

In [0]:
df5=dfr.select(dfr.id,dfr.name,json_tuple(dfr.marks,'telugu','english').alias('Telugu','English'))
df5.show()


+---+------+------+-------+
| id|  name|Telugu|English|
+---+------+------+-------+
|  1|lokesh|    25|     24|
+---+------+------+-------+



In [0]:
# we cam acces json objects from json string by using get_json_object easily with out doing above operations
from pyspark.sql.functions import get_json_object

In [0]:
dfr.select('name',get_json_object('marks','$.telugu').alias('Telugu')).show()

+------+------+
|  name|Telugu|
+------+------+
|lokesh|    25|
+------+------+

